In [322]:
import pandas as pd
import numpy as np
import logging

from pathlib import Path

import matplotlib.pyplot as plt
import mplleaflet
import folium

In [20]:
# Load data from matfile... oh noes...
matfile = r"p:\11204644-evaluatie-langsdammen\wp00_data_analyse\01_ADCP\output\mat\2016\0.2 Stroomrichting metingen\160629WL911500_langsdam1_af_Fle_0\160629WL911500_langsdam1_af_Fle_0_000_ASC.mat"


In [19]:
from scipy.io import loadmat, matlab
def load_mat(filename):
    """
    This function should be called instead of direct scipy.io.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    """

    def _check_vars(d):
        """
        Checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        """
        for key in d:
            if isinstance(d[key], matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
            elif isinstance(d[key], np.ndarray):
                d[key] = _toarray(d[key])
        return d

    def _todict(matobj):
        """
        A recursive function which constructs from matobjects nested dictionaries
        """
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _toarray(elem)
            else:
                d[strg] = elem
        return d

    def _toarray(ndarray):
        """
        A recursive function which constructs ndarray from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        """
        if ndarray.dtype != 'float64':
            elem_list = []
            for sub_elem in ndarray:
                if isinstance(sub_elem, matlab.mio5_params.mat_struct):
                    elem_list.append(_todict(sub_elem))
                elif isinstance(sub_elem, np.ndarray):
                    elem_list.append(_toarray(sub_elem))
                else:
                    elem_list.append(sub_elem)
            return np.array(elem_list)
        else:
            return ndarray

    data = loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_vars(data)

In [22]:
P = load_mat(matfile)

array_1D = []
array_2D = []
array_3D = []
for record in P['data_block']:

    params = {}
    arrays = {}
    subarray_3D = {}
    for k,v in record.items():
        if len(np.shape(v)) == 0:
            params[k] = v
        elif k == 'cords':
            params['X'] = v[0]
            params['Y'] = v[1]
        elif len(np.shape(v)) > 1:
            subarray_3D[k] = v
        elif np.shape(v)[0] > 10:
            arrays[k] = v
        else:
            # Not supported
            pass
#             print(k, np.shape(v))
    
    array_1D.append(params)
    array_2D.append(arrays)
    array_3D.append(subarray_3D)

array_1D = pd.DataFrame(array_1D)
array_2D = pd.concat({ii: pd.DataFrame(a) for ii, a in enumerate(array_2D)}, axis=1)
array_2D = array_2D.swaplevel(axis=1).sort_index(axis=1)

In [321]:
def create_geojson_object_from_matplotlib(d, color):
    fig, ax = plt.subplots()

    kw = dict(color=color, alpha=0.8, scale=10, cmap='jet')
    

    q = ax.quiver(d.X, d.Y, d.veast, d.vnorth,**kw)

    plt.axis('equal')
    
    gj = mplleaflet.fig_to_geojson(fig=fig)
    plt.close()
    return gj

layers = [2, 7]
colors = ['red', 'blue']
spacing = 50 # m


gjs = {}
for l, c in zip(layers, colors):
    
    # Merge data, drop na, and only show every [spacing] meters
    d = pd.concat([array_1D, array_2D.loc[l].unstack(level=0)], axis=1).dropna(how='any')
    every_spacing_m = d['distance'] // spacing
    ii = every_spacing_m.diff() == 1
    ii[0] = True
    d = d.loc[ii]
    
    gj = create_geojson_object_from_matplotlib(d, c)
   
    depth = d['depth'].mean()
    gjs[f'{depth:.2f} m'] = gj


In [ ]:
def add_gj_to_feature_group(gj, feature_group0):
    for feature in gj['features']:
        if feature['geometry']['type'] == 'Point':
            lon, lat = feature['geometry']['coordinates']
            div = feature['properties']['html']

            icon_anchor = (feature['properties']['anchor_x'],
                           feature['properties']['anchor_y'])

            icon = folium.features.DivIcon(html=div,
                                           icon_anchor=icon_anchor)
            marker = folium.Marker([lat, lon], icon=icon)
            feature_group0.add_child(marker)
        else:
            msg = "Unexpected geometry {}".format
            raise ValueError(msg(feature['geometry']))

In [326]:
mapa = folium.Map(location=[array_1D.Y.mean(), array_1D.X.mean()], zoom_start=14, tiles=None)

folium.TileLayer('OpenStreetMap', name=None, opacity=0.6, control=False).add_to(mapa)
lgd_txt = '<span style="color: {col};">{txt}</span>'

for (depth, gj), color in zip(gjs.items(), colors):
    
    feature_group = folium.FeatureGroup(name=lgd_txt.format(txt=f'Velocity at -{depth} m', col=color))
    add_gj_to_feature_group(gj, feature_group)
    mapa.add_child(feature_group)

mapa.add_child(folium.map.LayerControl(collapsed = False))



In [327]:
mapa.save('Test processing ADCP.html')